In [14]:
# Install required packages
!pip install SpeechRecognition pydub numpy

In [17]:
# 🧮 Voice-Controlled Arithmetic Calculator - Final Working Version

from IPython.display import Javascript, display
from google.colab import output
import base64, os
from pydub import AudioSegment
import speech_recognition as sr
import re

# Install dependencies
os.system('apt-get -qq install -y ffmpeg')
os.system('pip -q install pydub SpeechRecognition')

# JavaScript to record 3 seconds of mono audio
JS_RECORD = """
async function recordAudio() {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const ctx = new AudioContext();
  const src = ctx.createMediaStreamSource(stream);
  const dest = ctx.createMediaStreamDestination();
  src.connect(dest);

  const streamLeft = new MediaStream([dest.stream.getAudioTracks()[0]]);
  const mediaRecorder = new MediaRecorder(streamLeft);

  const chunks = [];
  mediaRecorder.ondataavailable = e => chunks.push(e.data);
  mediaRecorder.start();

  await new Promise(r => setTimeout(r, 3000));
  mediaRecorder.stop();

  return new Promise(resolve => {
    mediaRecorder.onstop = () => {
      const blob = new Blob(chunks, { type: 'audio/webm' });
      const reader = new FileReader();
      reader.onload = () => resolve(reader.result);
      reader.readAsDataURL(blob);
    };
  });
}
"""

def recognize_speech():
    print("🎙 Speak now! (Recording 3 seconds...)")

    # Inject JS and trigger recording
    display(Javascript(JS_RECORD))
    display(Javascript('recordAudio()'))

    # Wait for result
    try:
        data_b64 = output.eval_js("recordAudio()")
        header, data = data_b64.split(',', 1)
        webm_path = "rec.webm"
        with open(webm_path, "wb") as f:
            f.write(base64.b64decode(data))

        # Convert WebM to WAV
        wav_path = "rec.wav"
        AudioSegment.from_file(webm_path).set_frame_rate(16000).set_channels(1).export(wav_path, format="wav")

        # Recognize speech
        r = sr.Recognizer()
        with sr.AudioFile(wav_path) as source:
            audio = r.record(source)
        text = r.recognize_google(audio)
        print("🧠 Recognized:", text)

        # Parse expression
        expr = re.sub(r'(plus|times|multiplied by)', '*', text.lower())
        expr = re.sub(r'(minus|subtracted by)', '-', expr)
        expr = re.sub(r'(divided by|over)', '/', expr)
        expr = re.sub(r'[^\d\+\-\*/]', '', expr)

        if re.match(r'\d+[\+\-\*/]\d+', expr):
            result = eval(expr)
            print(f"✅ Result: {result}")
        else:
            print("❌ Could not parse expression.")

    except Exception as e:
        print(f"⚠ Error: {str(e)}")

# ▶ Run voice calculator
recognize_speech()

🎙 Speak now! (Recording 3 seconds...)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🧠 Recognized: what is 5 * 7
✅ Result: 35
